In [1]:
import folium

print(folium.__version__)

0.10.1


# 显示各级别地图

In [2]:
# define the world map
world_map = folium.Map()
# display world map
world_map

In [3]:
# define the national map
national_map = folium.Map(location=[35.3, 100.6], zoom_start=4)
# display national map
national_map

In [4]:
# define the city map
city_map = folium.Map(location=[39.93, 116.40], zoom_start=10)
# display city map
city_map

# 改变地图显示效果


In [5]:
# define the city map, tiles='Stamen Toner'
city_map = folium.Map(location=[39.93, 116.40], zoom_start=10, tiles='Stamen Toner')
# display city map
city_map

In [6]:
# define the city map, tiles='Stamen Terrain'
city_map = folium.Map(location=[39.93, 116.40], zoom_start=10, tiles='Stamen Terrain')
# display city map
city_map

# 在地图上标记

In [7]:
bj_map = folium.Map(location=[39.93, 115.40], zoom_start=12, tiles='Stamen Terrain')

folium.Marker(
    location=[39.95, 115.33],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(bj_map)

folium.Marker(
    location=[39.96, 115.32],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(bj_map)

folium.Marker(
    location=[39.93, 115.34],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(bj_map)

bj_map

In [8]:
bj_map = folium.Map(location=[39.93, 116.40], zoom_start=12, tiles='Stamen Toner')

folium.Circle(
    radius=200,
    location=[39.92, 116.43],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(bj_map)

folium.CircleMarker(
    location=[39.93, 116.38],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(bj_map)

bj_map

In [9]:
m = folium.Map(location=[46.1991, -122.1889],tiles='Stamen Terrain',zoom_start=13)

m.add_child(folium.LatLngPopup())

m

# 绘制热力图

In [10]:
# generated data
import numpy as np
data = (
    np.random.normal(size=(100, 3)) *
    np.array([[0.1, 0.1, 0.1]]) +
    np.array([[40, 116.5, 1]])
).tolist()
data[:3]

[[39.98827018368257, 116.68181362342246, 1.0112622486453433],
 [39.9429308757028, 116.47697751313335, 1.1359878437151856],
 [40.05835511717461, 116.57934685592211, 1.0546817212263948]]

In [11]:
from folium.plugins import HeatMap
m = folium.Map([39.93, 116.38], tiles='stamentoner', zoom_start=10)
# HeatMap
HeatMap(data).add_to(m)
# m.save(os.path.join('results', 'Heatmap.html'))
m

In [12]:
from folium.plugins import MarkerCluster

m = folium.Map([39.93, 116.38], tiles='stamentoner', zoom_start=10)

# create a mark cluster object
marker_cluster = MarkerCluster().add_to(m)

# add data point to the mark cluster
for lat, lng, label in data:
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(marker_cluster)

# add marker_cluster to map
m.add_child(marker_cluster)

In [13]:
import json
import requests

# read us-states border 
with open("us-states.json") as f:
    us_states = json.load(f)

us_map = folium.Map(location=[35.3, -97.6], zoom_start=4)
folium.GeoJson(
    us_states,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(us_map)

#display map
us_map

In [16]:
import geopandas as gpd
import pandas as pd
import fiona,folium, branca

states = gpd.GeoDataFrame.from_features(us_states, crs=fiona.crs.from_epsg(4326))
states.head()

,geometry,name
0,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska
2,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas
4,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",California


In [17]:
abbrs = pd.read_json(open("abbrs.json"))
abbrs.head(3)

,name,alpha-2
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


In [18]:
statesmerge = states.merge(abbrs,how='left', left_on='name', right_on='name')
statesmerge['geometry']=statesmerge.geometry.simplify(.05)
statesmerge.head()

,geometry,name,alpha-2
0,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama,AL
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska,AK
2,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona,AZ
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas,AR
4,"POLYGON ((-123.23326 42.00619, -120.00186 41.9...",California,CA


In [19]:
income = pd.read_csv("income.csv", dtype={"fips":str})
income['income-2015']=pd.to_numeric(income['income-2015'], errors='coerce')
income.groupby(by="state")[['state','income-2015']].median().head()
statesmerge['medianincome']=statesmerge.merge(income.groupby(by="state")[['state','income-2015']].median(), how='left', left_on='alpha-2', right_on='state')['income-2015']
statesmerge['change']=statesmerge.merge(income.groupby(by="state")[['state','change']].median(), how='left', left_on='alpha-2', right_on='state')['change']
statesmerge.head()

,geometry,name,alpha-2,medianincome,change
0,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama,AL,38721.5,2.779114
1,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska,AK,62561.5,14.758367
2,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona,AZ,43810.0,NaN
3,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas,AR,37890.0,10.897394
4,"POLYGON ((-123.23326 42.00619, -120.00186 41.9...",California,CA,53341.0,6.716596


In [20]:
# define colormap
colormap = branca.colormap.LinearColormap(vmin=statesmerge['change'].quantile(0.0), 
                                        vmax=statesmerge['change'].quantile(1), 
                                        colors=['darkgreen','green','lightblue','orange','red'],
                                       caption="State Level Median County Household Income (%)")

m = folium.Map(location=[35.3, -97.6], zoom_start=4)

# Mouse over display
tooltip=folium.features.GeoJsonTooltip(
    fields=["name", "medianincome", "change"],
    aliases=["State:", "2015 Median Income(USD):", "Median % Change:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """)


g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["change"])
        if x["properties"]["change"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    highlight_function=lambda x: {
        "fillColor": colormap(x["properties"]["change"])
        if x["properties"]["change"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.7,
    },
    tooltip=tooltip
).add_to(m)

colormap.add_to(m)

m